### HTTP client
start a builtin server by 'python -m http.server 8000'

(Explain why I don't teach writing a http server!)

In [1]:
import socket

HOST = '127.0.0.1'
PORT = 8000
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'GET /index.html HTTP/1.1\r\n\r\n')
    data = s.recv(1024)

In [2]:
print(data)

b'HTTP/1.0 200 OK\r\nServer: SimpleHTTP/0.6 Python/3.6.0\r\nDate: Mon, 13 Mar 2017 07:40:21 GMT\r\nContent-type: text/html\r\nContent-Length: 5\r\nLast-Modified: Mon, 06 Mar 2017 08:20:59 GMT\r\n\r\n'


In [3]:
tokens = data.split(b"\r\n")

import pprint as pp
pp.pprint(tokens)

for t in tokens:
    print(t.decode("utf-8"))

[b'HTTP/1.0 200 OK',
 b'Server: SimpleHTTP/0.6 Python/3.6.0',
 b'Date: Mon, 13 Mar 2017 07:40:21 GMT',
 b'Content-type: text/html',
 b'Content-Length: 5',
 b'Last-Modified: Mon, 06 Mar 2017 08:20:59 GMT',
 b'',
 b'']
HTTP/1.0 200 OK
Server: SimpleHTTP/0.6 Python/3.6.0
Date: Mon, 13 Mar 2017 07:40:21 GMT
Content-type: text/html
Content-Length: 5
Last-Modified: Mon, 06 Mar 2017 08:20:59 GMT




In [4]:
# Try CGI and multiple rcv()

''' /cgi-bin/mycgi.py can be as simple as:
#!/usr/bin/env python
print("Content-Type: text/html)
print()
print("<html><body>CGI</body></html>")
'''

import socket

def recvall(sock, buffer_size = 4096):
    buf = sock.recv(buffer_size)
    while buf:
        yield buf
        buf = sock.recv(buffer_size)

HOST = '127.0.0.1'
PORT = 8000
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'GET /cgi-bin/mycgi.py HTTP/1.1\r\n\r\n')
    
    response = b''.join(recvall(s))

In [5]:
print(response)

b'HTTP/1.0 200 Script output follows\r\nServer: SimpleHTTP/0.6 Python/3.6.0\r\nDate: Mon, 13 Mar 2017 07:40:21 GMT\r\nContent-type: text/html\r\n\r\n<h3>Input</h3>\r\n<h3>Environment</h3>\r\n<b>ALLUSERSPROFILE</b>: C:\\ProgramData</br>\r\n<b>APPDATA</b>: C:\\Users\\hsiao\\AppData\\Roaming</br>\r\n<b>COMMONPROGRAMFILES</b>: C:\\Program Files\\Common Files</br>\r\n<b>COMMONPROGRAMFILES(X86)</b>: C:\\Program Files (x86)\\Common Files</br>\r\n<b>COMMONPROGRAMW6432</b>: C:\\Program Files\\Common Files</br>\r\n<b>COMPUTERNAME</b>: DESKTOP-7M5J7JB</br>\r\n<b>COMSPEC</b>: C:\\WINDOWS\\system32\\cmd.exe</br>\r\n<b>CONDA_DEFAULT_ENV</b>: C:\\Users\\hsiao\\Miniconda3</br>\r\n<b>CONDA_PREFIX</b>: C:\\Users\\hsiao\\Miniconda3</br>\r\n<b>CONDA_PS1_BACKUP</b>: $P$G</br>\r\n<b>HOMEDRIVE</b>: C:</br>\r\n<b>HOMEPATH</b>: \\Users\\hsiao</br>\r\n<b>LOCALAPPDATA</b>: C:\\Users\\hsiao\\AppData\\Local</br>\r\n<b>LOGONSERVER</b>: \\\\DESKTOP-7M5J7JB</br>\r\n<b>NUMBER_OF_PROCESSORS</b>: 4</br>\r\n<b>ONEDRIVE</

### JSON
JSON (JavaScript Object Notation), specified by RFC 7159 (which obsoletes RFC 4627) and by ECMA-404, is a lightweight data interchange format inspired by JavaScript object literal syntax.

In [6]:
st = str("This is a string.")
li = list([1, 2., "st", (3, 4)])
di = {"one": 1, "two": 2, "three": 3}
tp = (6, 7, 8)

In [7]:
import json

print(json.dumps(st))
print(json.dumps(li))
print(json.dumps(di))
print(json.dumps(tp))

"This is a string."
[1, 2.0, "st", [3, 4]]
{"one": 1, "two": 2, "three": 3}
[6, 7, 8]


In [8]:
data  = (st, li, di, tp)

# Writing JSON data
with open('data.json', 'w') as f:
    json.dump(data, f)

# Reading data back
with open('data.json', 'r') as f:
    a, b, c, d = json.load(f)

print(a, b, c, d)

This is a string. [1, 2.0, 'st', [3, 4]] {'one': 1, 'two': 2, 'three': 3} [6, 7, 8]


### Base64

In [9]:
import base64
encoded = base64.b64encode(b'data to be encoded')
data = base64.b64decode(encoded)
print(encoded)
print(data)

b'ZGF0YSB0byBiZSBlbmNvZGVk'
b'data to be encoded'


### URL quote

In [10]:
import urllib.parse
q = urllib.parse.quote_plus('a&b/c=+-*/ abc:.')
print(q)
print(urllib.parse.unquote_plus(q))

a%26b%2Fc%3D%2B-%2A%2F+abc%3A.
a&b/c=+-*/ abc:.


### URLencode

In [11]:
import urllib.parse

parms = {
    'name1' : 'value1',
    'name2' : 'value2'
}

en = urllib.parse.urlencode(parms)
print(en)

print("GET /index.html?{}".format(en))

name1=value1&name2=value2
GET /index.html?name1=value1&name2=value2


### URL parse

In [12]:
import urllib.parse

url = r'https://docs.python.org/3.5/search.html?q=parse&check_keywords=yes&area=default'
parseResult = urllib.parse.urlparse(url)
print(parseResult)

param_dict = urllib.parse.parse_qs(parseResult.query)
print(param_dict)
print(param_dict['q']) # the return is a list. How to get plaintext 'parse'?

ParseResult(scheme='https', netloc='docs.python.org', path='/3.5/search.html', params='', query='q=parse&check_keywords=yes&area=default', fragment='')
{'q': ['parse'], 'check_keywords': ['yes'], 'area': ['default']}
['parse']


### urllib
In Python 2, use "urllib2"

In [13]:
import urllib.request
html = urllib.request.urlopen("http://127.0.0.1:8000/index.html").read(300)
print(html)

b'hello'


### GET with urllib

In [14]:
import urllib.request

parms = {
    'name1' : 'value1',
    'name2' : 'value2'
}

url = "http://127.0.0.1:8000/cgi-bin/mycgi.py"
q = urllib.parse.urlencode(parms)

html = urllib.request.urlopen("{}?{}".format(url, q)).read(4096).decode('utf-8')
print(html)

<h3>Input</h3>
<b>name1</b>: value1</br>
<b>name2</b>: value2</br>
<h3>Environment</h3>
<b>ALLUSERSPROFILE</b>: C:\ProgramData</br>
<b>APPDATA</b>: C:\Users\hsiao\AppData\Roaming</br>
<b>COMMONPROGRAMFILES</b>: C:\Program Files\Common Files</br>
<b>COMMONPROGRAMFILES(X86)</b>: C:\Program Files (x86)\Common Files</br>
<b>COMMONPROGRAMW6432</b>: C:\Program Files\Common Files</br>
<b>COMPUTERNAME</b>: DESKTOP-7M5J7JB</br>
<b>COMSPEC</b>: C:\WINDOWS\system32\cmd.exe</br>
<b>CONDA_DEFAULT_ENV</b>: C:\Users\hsiao\Miniconda3</br>
<b>CONDA_PREFIX</b>: C:\Users\hsiao\Miniconda3</br>
<b>CONDA_PS1_BACKUP</b>: $P$G</br>
<b>HOMEDRIVE</b>: C:</br>
<b>HOMEPATH</b>: \Users\hsiao</br>
<b>LOCALAPPDATA</b>: C:\Users\hsiao\AppData\Local</br>
<b>LOGONSERVER</b>: \\DESKTOP-7M5J7JB</br>
<b>NUMBER_OF_PROCESSORS</b>: 4</br>
<b>ONEDRIVE</b>: C:\Users\hsiao\OneDrive</br>
<b>OS</b>: Windows_NT</br>
<b>PATH</b>: C:\Users\hsiao\Miniconda3\Library\bin;C:\Users\hsiao\Miniconda3\Library\bin;C:\Users\hsiao\Miniconda3;C

### POST with urllib

In [15]:
import urllib.request
req = urllib.request.Request(url = 'http://127.0.0.1:8000/cgi-bin/mycgi.py',
                             data = b'data=This data is passed to stdin of the CGI',
                             method = 'POST')

with urllib.request.urlopen(req) as f:
    data = b''
    buf = f.read(300)
    data += buf
    while buf:
        buf = f.read(300)
        data += buf
    print(data.decode('utf-8'))

<h3>Input</h3>
<b>data</b>: This data is passed to stdin of the CGI</br>
<h3>Environment</h3>
<b>ALLUSERSPROFILE</b>: C:\ProgramData</br>
<b>APPDATA</b>: C:\Users\hsiao\AppData\Roaming</br>
<b>COMMONPROGRAMFILES</b>: C:\Program Files\Common Files</br>
<b>COMMONPROGRAMFILES(X86)</b>: C:\Program Files (x86)\Common Files</br>
<b>COMMONPROGRAMW6432</b>: C:\Program Files\Common Files</br>
<b>COMPUTERNAME</b>: DESKTOP-7M5J7JB</br>
<b>COMSPEC</b>: C:\WINDOWS\system32\cmd.exe</br>
<b>CONDA_DEFAULT_ENV</b>: C:\Users\hsiao\Miniconda3</br>
<b>CONDA_PREFIX</b>: C:\Users\hsiao\Miniconda3</br>
<b>CONDA_PS1_BACKUP</b>: $P$G</br>
<b>HOMEDRIVE</b>: C:</br>
<b>HOMEPATH</b>: \Users\hsiao</br>
<b>LOCALAPPDATA</b>: C:\Users\hsiao\AppData\Local</br>
<b>LOGONSERVER</b>: \\DESKTOP-7M5J7JB</br>
<b>NUMBER_OF_PROCESSORS</b>: 4</br>
<b>ONEDRIVE</b>: C:\Users\hsiao\OneDrive</br>
<b>OS</b>: Windows_NT</br>
<b>PATH</b>: C:\Users\hsiao\Miniconda3\Library\bin;C:\Users\hsiao\Miniconda3\Library\bin;C:\Users\hsiao\Minico

In [16]:
# if you need customized HTTP headers
import urllib.request

# Extra headers
'''
If you are running as a CGI, you can't read the HTTP header directly,
but the web server put much of that information into environment variables.
'''
headers = {
    'User-agent' : 'none/ofyourbusiness', # readable header
    'Spam' : 'Eggs' # custmized header is not reaable for cgi 
}

req = urllib.request.Request(url = 'http://127.0.0.1:8000/cgi-bin/mycgi.py',
                             data = b'data=This data is passed to stdin of the CGI',
                             headers = headers,
                             method = 'POST')

with urllib.request.urlopen(req) as f:
    data = b''
    buf = f.read(300)
    data += buf
    while buf:
        buf = f.read(300)
        data += buf
    print(data.decode('utf-8'))

<h3>Input</h3>
<b>data</b>: This data is passed to stdin of the CGI</br>
<h3>Environment</h3>
<b>ALLUSERSPROFILE</b>: C:\ProgramData</br>
<b>APPDATA</b>: C:\Users\hsiao\AppData\Roaming</br>
<b>COMMONPROGRAMFILES</b>: C:\Program Files\Common Files</br>
<b>COMMONPROGRAMFILES(X86)</b>: C:\Program Files (x86)\Common Files</br>
<b>COMMONPROGRAMW6432</b>: C:\Program Files\Common Files</br>
<b>COMPUTERNAME</b>: DESKTOP-7M5J7JB</br>
<b>COMSPEC</b>: C:\WINDOWS\system32\cmd.exe</br>
<b>CONDA_DEFAULT_ENV</b>: C:\Users\hsiao\Miniconda3</br>
<b>CONDA_PREFIX</b>: C:\Users\hsiao\Miniconda3</br>
<b>CONDA_PS1_BACKUP</b>: $P$G</br>
<b>HOMEDRIVE</b>: C:</br>
<b>HOMEPATH</b>: \Users\hsiao</br>
<b>LOCALAPPDATA</b>: C:\Users\hsiao\AppData\Local</br>
<b>LOGONSERVER</b>: \\DESKTOP-7M5J7JB</br>
<b>NUMBER_OF_PROCESSORS</b>: 4</br>
<b>ONEDRIVE</b>: C:\Users\hsiao\OneDrive</br>
<b>OS</b>: Windows_NT</br>
<b>PATH</b>: C:\Users\hsiao\Miniconda3\Library\bin;C:\Users\hsiao\Miniconda3\Library\bin;C:\Users\hsiao\Minico

### requests
http://docs.python-requests.org

In [17]:
import requests

url = 'http://127.0.0.1:8000/cgi-bin/mycgi.py'
parms = {
   'name1' : 'value1',
   'name2' : 'value2'
}
headers = {
    'User-agent' : 'none/ofyourbusiness',
    'Spam' : 'Eggs'
}

resp = requests.post(url, data=parms, headers=headers)
text = resp.text # no need to perform multiple read(), and text is utf-8
content = resp.content # binary data

print(text)

<h3>Input</h3>
<b>name2</b>: value2</br>
<b>name1</b>: value1</br>
<h3>Environment</h3>
<b>ALLUSERSPROFILE</b>: C:\ProgramData</br>
<b>APPDATA</b>: C:\Users\hsiao\AppData\Roaming</br>
<b>COMMONPROGRAMFILES</b>: C:\Program Files\Common Files</br>
<b>COMMONPROGRAMFILES(X86)</b>: C:\Program Files (x86)\Common Files</br>
<b>COMMONPROGRAMW6432</b>: C:\Program Files\Common Files</br>
<b>COMPUTERNAME</b>: DESKTOP-7M5J7JB</br>
<b>COMSPEC</b>: C:\WINDOWS\system32\cmd.exe</br>
<b>CONDA_DEFAULT_ENV</b>: C:\Users\hsiao\Miniconda3</br>
<b>CONDA_PREFIX</b>: C:\Users\hsiao\Miniconda3</br>
<b>CONDA_PS1_BACKUP</b>: $P$G</br>
<b>HOMEDRIVE</b>: C:</br>
<b>HOMEPATH</b>: \Users\hsiao</br>
<b>LOCALAPPDATA</b>: C:\Users\hsiao\AppData\Local</br>
<b>LOGONSERVER</b>: \\DESKTOP-7M5J7JB</br>
<b>NUMBER_OF_PROCESSORS</b>: 4</br>
<b>ONEDRIVE</b>: C:\Users\hsiao\OneDrive</br>
<b>OS</b>: Windows_NT</br>
<b>PATH</b>: C:\Users\hsiao\Miniconda3\Library\bin;C:\Users\hsiao\Miniconda3\Library\bin;C:\Users\hsiao\Miniconda3;C

In [18]:
# Try public http server
import requests

resp = requests.head('http://www.google.com.tw/') # www.python.org
content_length = resp.headers['content-length']

print(resp.status_code, resp.reason)
print(resp.headers)
print()
print(resp.cookies) # cookies can be re-sent to next requests.get(), post(), ...

200 OK
{'Date': 'Mon, 13 Mar 2017 07:40:21 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=Big5', 'P3P': 'CP="This is not a P3P policy! See https://www.google.com/support/accounts/answer/151657?hl=en for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'Content-Length': '4624', 'X-XSS-Protection': '1; mode=block', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': 'NID=98=sfDWMO3WEozQ8x68sfbD4FSA1onxNFCOQv6VkAofFJlCs6jSAwMwwpl6XgQv1iXk2LWvL-DjfnZLDidPpIsrXRcKWR2_kqEct0uJaQpEE6fAnmzLSADy_asfKct0_8tt; expires=Tue, 12-Sep-2017 07:40:21 GMT; path=/; domain=.google.com.tw; HttpOnly'}

<RequestsCookieJar[<Cookie NID=98=sfDWMO3WEozQ8x68sfbD4FSA1onxNFCOQv6VkAofFJlCs6jSAwMwwpl6XgQv1iXk2LWvL-DjfnZLDidPpIsrXRcKWR2_kqEct0uJaQpEE6fAnmzLSADy_asfKct0_8tt for .google.com.tw/>]>


### http.client
if you need low level control ...

### SSL, HTTPS

In [19]:
import http.client
conn = http.client.HTTPSConnection("www.python.org")
conn.request("GET", "/")
r1 = conn.getresponse()
print(r1.status, r1.reason)

data1 = r1.read() # This will return entire content.
conn.close()

200 OK


In [20]:
# The following example demonstrates reading data in chunks.
import http.client
conn2 = http.client.HTTPSConnection("www.google.com.tw")
conn2.request("GET", "/")
r2 = conn2.getresponse()
body = r2.read(100)
while body:
    print(body)
    body = r2.read(100) # 1024 bytes
conn2.close()

b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="zh-TW"><head><meta cont'
b'ent="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/logos/doodles/2017/holi-fes'
b'tival-2017-5750729821126656-hp.gif" itemprop="image"><meta content="2017 \xa6L\xab\xd7\xc5x\xac\xf5\xb8` #GoogleDoodle" p'
b'roperty="og:description"><meta content="http://www.google.com/logos/doodles/2017/holi-festival-2017-'
b'5750729821126656-thp.png" property="og:image"><meta content="420" property="og:image:width"><meta co'
b'ntent="150" property="og:image:height"><meta content="http://www.google.com/logos/doodles/2017/holi-'
b'festival-2017-5750729821126656-hp2x.gif" property="og:url"><meta content="video.other" property="og:'
b'type"><title>Google</title><script>(function(){window.google={kEI:\'5kzGWLboOsLh0gSGvqG4Cw\',kEXPI:\'13'
b'52467,1352863,1352993,1353047,3700315,3700347,4029815,4031109,4032677,4036527,4038012,4039268,404349'
b'2,4045841,4048347,4065786,4066195,4071

In [21]:
import http.client, urllib.parse
params = urllib.parse.urlencode({'@number': 12524, '@type': 'issue', '@action': 'show'})
headers = {"Content-type": "application/x-www-form-urlencoded",
           "Accept": "text/plain"}

conn = http.client.HTTPConnection("bugs.python.org")
conn.request(method = "POST", url = "", body = params, headers = headers)
response = conn.getresponse()
print(response.status, response.reason)

data = response.read()
print(data)
conn.close()

302 Found
b'Redirecting to <a href="http://bugs.python.org/issue12524">http://bugs.python.org/issue12524</a>'
